In [ ]:
# 📂 Montar Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# ⬇️ Clonar Automatic1111 WebUI
!git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui /content/stable-diffusion-webui

In [ ]:
# 🔗 Linkar pasta de LoRAs
!mkdir -p /content/drive/MyDrive/sd_loras
!mkdir -p /content/stable-diffusion-webui/models/Lora
!ln -s /content/drive/MyDrive/sd_loras /content/stable-diffusion-webui/models/Lora/sd_loras

In [ ]:
# 🛠️ Corrigir erro dos tokenizers
!pip install tokenizers==0.13.3 --force-reinstall

In [ ]:
# ▶️ Rodar em modo CPU
%cd /content/stable-diffusion-webui
!COMMANDLINE_ARGS="--share --disable-safe-unpickle --skip-torch-cuda-test --precision full --no-half" \
  REQS_FILE="requirements.txt" \
  python launch.py

In [ ]:
# ⏳ Anti-Idle (mantém Colab ativo)
import time
while True:
    time.sleep(60)